# Data cleaning and pre-processing
### In this notebook, we clean and pre-process the raw dataset provided by Iowa State University. 

In [14]:
# Import data handling libraries
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add root directory to Python path
root_dir = Path.cwd().parent  # Go up one level from Notebooks folder
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Now import from Source Code directory
sys.path.append(str(root_dir / 'Source Code'))
from cleaner import *

#read in the raw dataset
df = pd.read_csv('../Data/Datasets/data_set_10_17.csv')

df.sample(5)

,STUDENT,MAJOR_CURR,MAJOR_COLL,CLSFN_YR,ENTRY_SEM_CD,ENTRY_CCYY,LEVEL,SEM_CCYY,SEM_CD,OFFER_DEPT_ABRVN,...,DEG_DATEDB,MAJOR_CURR.1,DEG_DESCR,LAST_RGST_TERM,CLSFN_YR.1,MAJOR_CURR.2,MAJOR_COLL.1,Workday Enrolled in Fall 2024,Workday Enrolled in Fall 2024 Class Standing,Workday Enrolled in Fall 2024 Primary Program of Study
6918,2856,GEOL,S,4,1,1991,U,2014,F,MATH,...,12/18/1999,PHYS,B.S. LIBERAL ARTS AND SCIENCES ...,F16,4.0,GEOL,S,NaN,NaN,NaN
15074,6462,MU BM,S,4,F,2017,U,2020,F,MATH,...,8/5/2023,MU BA,B.A. LIBERAL ARTS & SCIENCES ...,123,4.0,MU BM,S,NaN,NaN,NaN
2952,1233,PHYS,S,2,F,2022,U,2023,F,MATH,...,NaN,NaN,NaN,S24,3.0,PHYS,S,98243202.0,Junior,"Physics, B.S."
7962,3287,MATH,S,4,F,2017,U,2020,S,MATH,...,5/8/2021,MATH,B.S. LIBERAL ARTS & SCIENCES ...,S21,4.0,MATH,S,NaN,NaN,NaN
27897,11951,COM S,S,2,F,2020,U,2021,F,MATH,...,NaN,NaN,NaN,S22,3.0,COM S,S,NaN,NaN,NaN


Before we start with any data cleaning, let's first get a quick look at the columns and statistics in the dataset.

In [ ]:
print_column_summary(df)
gradestats(df)

Column Summary:
Column Name               Description                                       
---------------------------------------------------------------------------
STUDENT                   student identifier (integer)                      
MAJOR_CURR                major at time of taking the course                
MAJOR_COLL                College of liberal arts for all courses           
CLSFN_YR                  Classification of the year (Freshman, Sophomore, etc.) when course was taken
ENTRY_SEM_CD              Semester of enrollment ('F' --> Fall, 'S' --> Spring, '1' --> Summer)
ENTRY_CCYY                Year of enrollment                                
LEVEL                     Irrelevant, to be dropped                         
SEM_CCYY                  Year in which course was taken                    
SEM_CD                    Semester in which course was taken                
OFFER_DEPT_ABRVN          Same for all rows (Math department)               
CRSE            

Each student comes with a unique integer identifier. The rows correspond to each course taken by the student. In the final dataset, we will have only one row per student, and the columns will correspond to courses. So, we begin our cleaning with:

1. Drop unnecessary columns at the outset.

In [ ]:
df = drop_initial_cols(df)
df.columns

Index(['STUDENT', 'ENTRY_CCYY', 'ENTRY_SEM_CD', 'SEM_CCYY', 'SEM_CD',
       'SEM_CCYY.1', 'SEM_CD.1', 'DEG_CD', 'GRADE_CATGORY', 'CRSE',
       'CRSE_TITLE'],
      dtype='object')

2. Next, we format the date/semester columns for entry, course, and graduation. We will combine each pair into a single unique identifier (float), named `ENT_SEM`, `CRSE_SEM`, and `GRAD_SEM`.
    - For enrollment, we encode Spring, Summer and Fall as $0.0$, $0.4$, and $0.6$, respectively.   
    - For course and graduation, we encode Spring, Summer and Fall as $0.4$, $0.6$, and $1.0$, respectively.
    - The new columns are computed by the formula (year mod 2000) + (sem value)
    - Thus, if a student enrolled in Fall 2012, and took a particular course in Spring 2014, then `ENT_SEM`$ = 12.6$ and `CRSE_SEM`$ = 14.4$. 
    - Similarly, if they graduated in Spring 2016, then `GRAD_SEM`$ = 16.4$.

In [ ]:
df = format_dates(df)
df[['ENT_SEM','CRSE_SEM','GRAD_SEM']].sample(5)

,ENT_SEM,CRSE_SEM,GRAD_SEM
10678,17.6,18.4,22.4
8041,12.0,16.0,16.4
3800,22.6,23.0,NaN
21920,11.6,15.0,17.4
1965,22.6,23.0,NaN


3. Next, we drop students who enrolled before 2008 (because data is shaky), and also those who enrolled in Spring 2021 or later (since we cannot yet determine if these student will have graduated in 4 years or not).

In [ ]:
df = remove_by_ENT_SEM(df)

After removing students who entered before 2008:
Rows: 30219
Students: 13013
Grads: 7578
Grad rate: 58.23

After dropping students who enrolled in Spring 2021 or later:
Rows: 24855
Students: 10012
Grads: 7385
Grad rate: 73.76


4. Next, we drop all students who received a graduate degree. (Note, we completely drop the student even though they may have also received a bachelors degree. This is because it is not clear whether a particular course was taken towards the bachelors degree or graduate degree).

In [ ]:
df = remove_graduate_degrees(df)

After dropping students who took classes towards a graduate degree
Rows: 22362
Students: 9689
Grads: 7062
Grad rate: 72.89


5. Next, we drop all rows corresponding to courses taken after graduation. 
    
    We also re-format `CRSE_SEM` and `GRAD_SEM` to contain the timestamp relative to `ENT_SEM`, and we multiply by $2.5$ so that fall and spring count as 1 semester and Summer counts as half a semester (so a full year counts as $2.5$ semesters). 
    
    Our final target variable will be $1$ if the student "graduates in 4 years"-- for us, this means `GRAD_SEM` is at most $9.5$, which is the typical case when a student enrolls in Fall and graduates in Spring 4 years later.
    
    Accordingly, we won't consider courses taken after $9.5$ years, so we drop all rows where `CRSE_SEM` is greater than $9.5$.

In [ ]:
df = remove_by_CRSE_SEM(df)

After dropping courses taken after 9.5 semesters:
Rows: 21068
Students: 9401
Grads: 6821
Grad rate: 72.56

After removing courses taken after graduation:
Rows: 21059
Students: 9398
Grads: 6818
Grad rate: 72.55


6. We remove any duplicate rows at this point. (These exist because some of the original rows were repeated for each degree earned by the student.)

In [ ]:
df = remove_duplicates(df)

Number of duplicate rows: 1184
After dropping duplicates:
Rows: 19875
Students: 9398
Grads: 6818
Grad rate: 72.55


7. Next, we format the course numbers and titles, combine together similar courses, and drop courses that are no longer offered, or which have an overall low enrollment (<50).

In [ ]:
df = format_and_drop_CRSE(df)

After formatting courses and titles, and removing courses with low enrollment:
Rows: 18802
Students: 9181
Grads: 6647
Grad rate: 72.4


Before continuing with the cleaning, let's pause to make a csv with the remaining courses and their enrollment counts. We will store the courses in a list for the next step of cleaning.

In [ ]:
df, courses = make_crse_counts_csv(df)

Left with 28 courses: ['165', '166', '265', '143', '140', '104', '150', '207', '201', '317', '267', '105', '301', '266', '414', '160', '101', '25', '492', '341', '385', '435', '397', '436', '495', '314', '500', '304']


8. Next, we format the `GRADE_CATGORY` column, using the value $1$ for 'ABOVE C-' and 'Satisfactory/Pass', and $-1$ for 'C- OR BELOW'.

    Then, for each `x` in `courses` (list of all `CRSE` values appearing), we create a column which contains the grade ($\pm 1$) times the course semester (a float between $0.4$ and $3.8$). Thus, the values in `df[x]` will range from $-3.8$ to $3.8$.

In [ ]:
df = make_CRSE_columns(df,courses)

9. Finally, we remove the unnecessary columns and aggregate the dataset by student, taking the value in each column that is largest in absolute value (that is, we keep only the data of the last time a particular course was taken by a student). Finally, we add a column `Y` which equals 1 if `GRAD_SEM` is at most $3.8$, and $0$ otherwise. 

In [ ]:
final_cols = ['STUDENT','ENT_SEM','GRAD_SEM'] + courses
df = agg_and_add_Y(df,final_cols)

After aggregating the dataset by student and adding the target variable:
Rows: 9181
Students 9181
Grads: 4496
Grad rate: 48.97


10. We're done! So we now save to csv.

In [13]:
#save the final dataset
df.to_csv('../Data/Datasets/dataset_cleaned.csv', index=False)

At this point, the cleaning and pre-processing is done. We conclude this notebook by making dictionaries of the math major courses and general math courses.

In [14]:
crse_dict = \
{'165': 'CALCULUS I',
 '166': 'CALCULUS II',
 '265': 'CALCULUS III',
 '143': 'PREP FOR CALCULUS',
 '140': 'COLLEGE ALGEBRA',
 '104': 'INTRO TO PROBABILTY',
 '150': 'DISC MATH BUS&SOC S',
 '207': 'MATRCES&LINEAR ALGB',
 '201': 'INTRO TO PROOFS',
 '317': 'THRY LINEAR ALGEBRA',
 '267': 'DIFF EQ & TRANSFMS',
 '105': 'INTRO TO MATH IDEAS',
 '301': 'ABSTRACT ALGEBRA I',
 '266': 'ELEM DIFF EQUATNS',
 '414': 'ANALYSIS I',
 '160': 'SURVEY OF CALCULUS',
 '101': 'ORIENTATION IN MATH',
 '25': 'HIGH SCHOOL ALGEBRA',
 '492': 'UNDERGRAD SEMINAR',
 '341': 'INTR THY PROBAB&S I',
 '385': 'INTR PART DIFF EQUA',
 '435': 'GEOMETRY I',
 '397': 'TEACH SEC SCHL MATH',
 '436': 'GEOMETRY II',
 '495': 'INDEPENDENT STUDY',
 '314': 'GRAPH THEORY',
 '500': 'GRADUATE MATH',
 '304': 'COMBINATORICS'}

gen_dict = \
{'165': 'CALCULUS I',
 '166': 'CALCULUS II',
 '265': 'CALCULUS III',
 '143': 'PREP FOR CALCULUS',
 '140': 'COLLEGE ALGEBRA',
 '104': 'INTRO TO PROBABILTY',
 '150': 'DISC MATH BUS&SOC S',
 '207': 'MATRCES&LINEAR ALGB',
 '267': 'DIFF EQ & TRANSFMS',
 '105': 'INTRO TO MATH IDEAS',
 '160': 'SURVEY OF CALCULUS',
 '25': 'HIGH SCHOOL ALGEBRA'}

math_dict = \
{'201': 'INTRO TO PROOFS',
 '317': 'THRY LINEAR ALGEBRA',
 '301': 'ABSTRACT ALGEBRA I',
 '266': 'ELEM DIFF EQUATNS',
 '414': 'ANALYSIS I',
 '101': 'ORIENTATION IN MATH',
 '492': 'UNDERGRAD SEMINAR',
 '341': 'INTR THY PROBAB&S I',
 '385': 'INTR PART DIFF EQUA',
 '435': 'GEOMETRY I',
 '397': 'TEACH SEC SCHL MATH',
 '436': 'GEOMETRY II',
 '495': 'INDEPENDENT STUDY',
 '314': 'GRAPH THEORY',
 '500': 'GRADUATE MATH',
 '304': 'COMBINATORICS'}

#save the dictionaries
import json
with open('../Data/Dictionaries/crse_dict.json', 'w') as fp:
    json.dump(crse_dict, fp)
with open('../Data/Dictionaries/gen_dict.json', 'w') as fp:
    json.dump(gen_dict, fp)
with open('../Data/Dictionaries/math_dict.json', 'w') as fp:
    json.dump(math_dict, fp)